In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#Need to know the number of unique values in the column in order to know what name to assign

In [ ]:
# importing one hot encoder from sklearn
# There are changes in OneHotEncoder class
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

from six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
'''
Aim is to design a decision tree classifier based on the fact if the
solver produces an output or not
'''
summary = pd.read_csv('/home/skapoor/Thesis/gmwcs-solver/outputs/solver/summary_100.csv')
summary.fillna(0, inplace=True)

In [ ]:
summary.sort_values(['Features_preserved_per', 'Output_graph_posedge'], ascending=[False, False])
filtered = summary[(summary.Features_preserved_per>0) & (summary.Output_graph_posedge>0) & (summary.Output_Graph_edges>4) ]
#the filtered valuesa are actually what we want
# the decision tree will be first run on the basis of the first six columns and the targets are number of edges in binarized

In [ ]:
for target in summary['Target'].unique():
	df = summary[summary['Target']==target].iloc[:,:6]
	y = summary[summary['Target'] == target].Features_preserved_per
	y = y>0
# encode all the categorical variables
	le = LabelEncoder()
	df.Feature_type = le.fit_transform(df.Feature_type)
	c1 = list(le.classes_)
	df.Edge = le.fit_transform(df.Edge)
	c3 = list(le.classes_)
	df.Node_weights = le.fit_transform(df.Node_weights)
	c4 = list(le.classes_)
	columnTransformer = ColumnTransformer([('encoder',
											OneHotEncoder(),
											[0,3,4])],
										remainder='passthrough')
	df.drop_columns(['Target'])
	data = np.array(columnTransformer.fit_transform(df), dtype=np.str)
	c1.extend(['Factor'])
	c1.extend(c3)
	c1.extend(c4)
	c1.append('Subtracted_value')
	data = pd.DataFrame(data, columns=c1)
	dtree=DecisionTreeClassifier()
	dtree.fit(data,y)

	dot_data = StringIO()
	export_graphviz(dtree, out_file=dot_data,
					filled=True, rounded=True,
					special_characters=True)
	graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
	Image(graph.create_png())
    export_graphviz(dtree, out_file=f'{target}.dot', feature_names=data.columns)